In [0]:
%pip install -r ../../requirements.txt

In [0]:
catalog_name = 'dbxmetagen'
schema_name = 'default'
table_name = 'healthcare_test'

spark.sql(f"""
SELECT t.constraint_catalog, 
t.constraint_schema, 
c.table_name, 
c.constraint_name, 
t.constraint_type, 
c.column_name
FROM system.information_schema.table_constraints t
LEFT JOIN system.information_schema.constraint_column_usage c
        ON t.constraint_catalog = c.constraint_catalog 
        AND t.constraint_schema = c.constraint_schema 
        AND t.constraint_name = c.constraint_name
WHERE t.constraint_catalog = '{catalog_name}'
AND t.constraint_schema = '{schema_name}'
AND t.table_name = '{table_name}';
""").display()

In [0]:
%sql
SELECT * FROM dbxmetagen.metadata_results.pi_metadata_generation_log ORDER BY _created_at DESC;

In [0]:
%sql
SELECT * FROM dbxmetagen.metadata_results.table_processing_log ORDER BY `_updated_at` DESC;

In [0]:
# Create the Delta table with room_number as a primary key
spark.sql("""
CREATE OR REPLACE TABLE dbxmetagen.default.rooms (
    room_number STRING,
    room_name STRING,
    capacity INT,
    PRIMARY KEY (room_number)
) USING DELTA
""")

# Insert 3 rows into the rooms table
spark.sql("""
INSERT INTO dbxmetagen.default.rooms (room_number, room_name, capacity) VALUES
('101A', 'Room A', 2),
('102', 'Room B', 4),
('103', 'Room C', 3)
""")

# First, alter the column to be non-nullable
try:
    spark.sql("""
    ALTER TABLE dbxmetagen.default.healthcare_test
    ALTER COLUMN patient_id SET NOT NULL
    """)

    # Then, add the primary key constraint
    spark.sql("""
    ALTER TABLE dbxmetagen.default.healthcare_test
    ADD CONSTRAINT patient_pk PRIMARY KEY (patient_id)
    """)
except: 
    print("already added or didnt work")

# Finally, add the foreign key constraint
spark.sql("""
ALTER TABLE dbxmetagen.default.healthcare_test
ADD CONSTRAINT room_number_fk
FOREIGN KEY (room_number) REFERENCES dbxmetagen.default.rooms
""")

In [0]:
from src.dbxmetagen.deterministic_pi import detect_pi
from presidio_analyzer import AnalyzerEngine

analyzer = AnalyzerEngine()
all_entities = analyzer.get_supported_entities(language="en")
print(all_entities)